In [1]:
"""Minimal jobshop example."""
import collections
from ortools.sat.python import cp_model


In [2]:
# Job情報
jobs_data = [  # task = (machine_id, processing_time).
    [(0, 3), (1, 2), (2, 2)],  # Job0
    [(0, 2), (2, 1), (1, 4)],  # Job1
    [(1, 4), (2, 3)],  # Job2
    [(1, 2), (0, 1), (2, 4)],  # Job3
    [(2, 1), (0, 2), (1, 1)],  # Job4
]

# Machineの数
machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

# Taskの合計時間
horizon = sum(task[1] for job in jobs_data for task in job)

# モデルの作成
model = cp_model.CpModel()

In [3]:
# Taskの種類
task_type = collections.namedtuple('task_type', 'start end interval')

# 解取り回し用
assigned_task_type = collections.namedtuple('assigned_task_type',
                                            'start job index duration')

# ジョブタスクの処理時間
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

# 開始、終了、処理時間変数を作成
for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                            'interval' + suffix)
        all_tasks[job_id, task_id] = task_type(start=start_var,
                                               end=end_var,
                                               interval=interval_var)
        machine_to_intervals[machine].append(interval_var)


In [4]:
# 重複無の制約を追加
for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

# 実行順序の制約を追加
for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id +
                            1].start >= all_tasks[job_id, task_id].end)

In [10]:
# 目的関数
obj_var = model.NewIntVar(0, horizon, 'makespan')
model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
model.Minimize(obj_var)


solver = cp_model.CpSolver()
status = solver.Solve(model)


# Statistics.
print("\nStatistics")
print(f"  - conflicts: {solver.NumConflicts()}")
print(f"  - branches : {solver.NumBranches()}")
print(f"  - wall time: {solver.WallTime()}s")


Statistics
  - conflicts: 1
  - branches : 25
  - wall time: 0.005227561s


In [11]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('Solution:')
    # Machine毎に割当てられたTaskのリストを作成
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(start=solver.Value(
                    all_tasks[job_id, task_id].start),
                                   job=job_id,
                                   index=task_id,
                                   duration=task[1]))

    # Machine毎のOutputを作成
    output = ''
    for machine in all_machines:
        # 開始時間でソート
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Machine ' + str(machine) + ': '
        sol_line = '           '

        for assigned_task in assigned_jobs[machine]:
            name = 'job_%i_task_%i' % (assigned_task.job,
                                       assigned_task.index)
            # スペース挿入
            sol_line_tasks += '%-15s' % name

            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            # スペース挿入
            sol_line += '%-15s' % sol_tmp

        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
    print(output)
else:
    print('No solution found.')

Solution:
Optimal Schedule Length: 13.0
Machine 0: job_1_task_0   job_3_task_1   job_0_task_0   job_4_task_1   
           [0,2]          [2,3]          [3,6]          [6,8]          
Machine 1: job_3_task_0   job_2_task_0   job_0_task_1   job_4_task_2   job_1_task_2   
           [0,2]          [2,6]          [6,8]          [8,9]          [9,13]         
Machine 2: job_4_task_0   job_1_task_1   job_3_task_2   job_2_task_1   job_0_task_2   
           [0,1]          [2,3]          [3,7]          [7,10]         [10,12]        

